# Polymer Builder: Three-Layer Architecture - Complete Guide

The polymer builder uses a three-layer architecture to separate concerns: monomer selection, chain generation, and system planning. This guide covers all components and usage patterns.

## Table of Contents

1. [Architecture Overview](#architecture)
2. [Layer 1: SequenceGenerator](#sequence)
3. [Layer 2: PolydisperseChainGenerator](#chain)
4. [Layer 3: SystemPlanner](#system)
5. [DP Distributions](#distributions)
6. [Complete Examples](#examples)
7. [Advanced Topics](#advanced)

## Architecture Overview

### Three Layers

```
SystemPlanner                  # Top: system constraints
  ↓ calls
PolydisperseChainGenerator     # Middle: chain distributions
  ↓ calls
SequenceGenerator              # Bottom: monomer selection
```

### Separation of Concerns

**Why three layers?**

1. **Testability**: Each layer independently testable
2. **Reusability**: Mix and match components
3. **Clarity**: One responsibility per layer

### Layer Responsibilities

| Layer | Input | Output | Controls |
|-------|-------|--------|----------|
| SequenceGenerator | DP (int) | Sequence (list[str]) | Monomer probabilities |
| PolydisperseChainGenerator | RNG | Chain (DP, sequence, mass) | Chain length distribution |
| SystemPlanner | Target mass | SystemPlan (list[Chain]) | System constraints |

## Layer 1: SequenceGenerator

### WeightedSequenceGenerator

Selects monomers based on relative weights.

In [1]:
from molpy.builder.polymer import WeightedSequenceGenerator
from random import Random

# 70% A, 30% B random copolymer
seq_gen = WeightedSequenceGenerator(
    monomer_weights={'A': 0.7, 'B': 0.3}
)

# Generate sequence
rng = Random(42)
sequence = seq_gen.generate_sequence(dp=20, rng=rng)

print(f"Sequence: {sequence}")
print(f"Composition: A={sequence.count('A')}, B={sequence.count('B')}")

Sequence: ['A', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A']
Composition: A=17, B=3


### Expected Composition

Get theoretical monomer fractions.

In [2]:
expected = seq_gen.expected_composition()
print("Expected composition:")
for monomer, fraction in expected.items():
    print(f"  {monomer}: {fraction:.2%}")

Expected composition:
  A: 70.00%
  B: 30.00%


## Layer 2: PolydisperseChainGenerator

Samples chain length from distribution, generates sequence, computes mass.

In [3]:
from molpy.builder.polymer import PolydisperseChainGenerator, SchulzZimmDPDistribution

# Schulz-Zimm distribution (PDI = 2.0)
dp_dist = SchulzZimmDPDistribution(
    Mn=10000, Mw=20000, avg_monomer_mass=100
)

# Chain generator
chain_gen = PolydisperseChainGenerator(
    seq_generator=seq_gen,
    monomer_mass={'A': 100, 'B': 120},
    dp_distribution=dp_dist
)

# Build chain
chain = chain_gen.build_chain(Random(42))
print(f"Chain: DP={chain.dp}, mass={chain.mass:.0f} g/mol")

Chain: DP=11, mass=1140 g/mol


## Layer 3: SystemPlanner

Generates multiple chains to meet target mass.

In [4]:
# NOTE: This example is temporarily disabled due to API changes
# TODO: Update to current API

# from molpy.builder.polymer import SystemPlanner
# 
# # System planner
# planner = SystemPlanner(chain_gen)
# plan = planner.plan_system(target_mass=100000, random_seed=42)
# 
# print(f"System Plan:")
# print(f"  Chains: {len(plan.chains)}")
# print(f"  Total mass: {plan.total_mass:.0f} g/mol")
# print(f"  Target: {plan.target_mass:.0f} g/mol")

## DP Distributions

### SchulzZimmDPDistribution

Gamma distribution for polydisperse systems.

**Parameters:**
- `Mn`: Number-average MW
- `Mw`: Weight-average MW
- `avg_monomer_mass`: Average monomer mass

**Derived:**
- `PDI = Mw/Mn`
- Shape parameter `k = Mn/(Mw-Mn)`
- Scale parameter `θ = (Mw-Mn)/Mn`

## See Also

- [Sequence Generator](sequence_generator.ipynb): Detailed sequence generation
- [Polydisperse Chain Generator](polydisperse_chain_generator.ipynb): Chain distributions
- [System Planner](system_planner.ipynb): System planning